# Day 5: Supply Stacks
## Part One

The expedition can depart as soon as the final supplies have been unloaded from the ships. Supplies are stored in stacks of marked _crates_, but because the needed supplies are buried under many other crates, the crates need to be rearranged.

The ship has a _giant cargo crane_ capable of moving crates between stacks. To ensure none of the crates get crushed or fall over, the crane operator will rearrange them in a series of carefully-planned steps. After the crates are rearranged, the desired crates will be at the top of each stack.

The Elves don't want to interrupt the crane operator during this delicate procedure, but they forgot to ask her _which_ crate will end up where, and they want to be ready to unload them as soon as possible so they can embark.

They do, however, have a drawing of the starting stacks of crates _and_ the rearrangement procedure (your puzzle input). For example:

```
    [D]    
[N] [C]    
[Z] [M] [P]
 1   2   3 

move 1 from 2 to 1
move 3 from 1 to 3
move 2 from 2 to 1
move 1 from 1 to 2
```

In this example, there are three stacks of crates. Stack 1 contains two crates: crate `Z` is on the bottom, and crate `N` is on top. Stack 2 contains three crates; from bottom to top, they are crates `M`, `C`, and `D`. Finally, stack 3 contains a single crate, `P`.

Then, the rearrangement procedure is given. In each step of the procedure, a quantity of crates is moved from one stack to a different stack. In the first step of the above rearrangement procedure, one crate is moved from stack 2 to stack 1, resulting in this configuration:

```
[D]        
[N] [C]    
[Z] [M] [P]
 1   2   3 
```

In the second step, three crates are moved from stack 1 to stack 3. Crates are moved _one at a time_, so the first crate to be moved (`D`) ends up below the second and third crates:

```
        [Z]
        [N]
    [C] [D]
    [M] [P]
 1   2   3
```

Then, both crates are moved from stack 2 to stack 1. Again, because crates are moved _one at a time_, crate `C` ends up below crate `M`:

```
        [Z]
        [N]
[M]     [D]
[C]     [P]
 1   2   3
```

Finally, one crate is moved from stack 1 to stack 2:

```
        [Z]
        [N]
        [D]
[C] [M] [P]
 1   2   3
```

The Elves just need to know _which crate will end up on top of each stack_; in this example, the top crates are `C` in stack 1, `M` in stack 2, and `Z` in stack 3, so you should combine these together and give the Elves the message `_CMZ_`.

_After the rearrangement procedure completes, what crate ends up on top of each stack?_

### Functions

In [1]:
def parse_movements(fn: str, no_header_lines: int) -> list[tuple[int,int,int]]:
    def parse_move(line: str) -> tuple[int,int,int]:
        # move N from X to Y
        elems = line.split()
        n,x,y = int(elems[1]), int(elems[3]), int(elems[5])
        return n,x,y

    with open(fn, 'r') as f:
        input = f.readlines()
        # skip header
        input = input[no_header_lines:]
        # parse movements
        commands = [parse_move(line) for line in input]
        return commands

In [2]:
def print_stacks(stacks: list[list[str]]):
    max_len = max([len(l) for l in stacks])
    for i in reversed(range(max_len)):
        line_items = []
        for stack in stacks:
            try:
                item = stack[i]
                line_items.append(f"[{item}]")
            except IndexError:
                line_items.append("   ")
                continue
        out_str = " ".join(line_items)
        print(out_str)
    indices = list(range(1,len(stacks)+1))
    indices = [f" {i} " for i in indices]
    print(" ".join(indices))

In [3]:
def execute_move_fifo(stacks: list[list[str]], move: tuple[int,int,int]):
    num, from_, to_ = move
    # indices start at 1 so move -1
    from_ -= 1
    to_ -= 1
    cache = []
    # FIFO
    # remove one after the other
    for _ in range(num):
        cache.append(stacks[from_].pop())
    # add to new destination
    for _ in range(len(cache)):
        stacks[to_].append(cache.pop(0))
    return

In [4]:
def task1(stacks: list[list[str]]):
    out_str = "".join([l.pop() for l in stacks])
    return out_str

### Example Input

In [5]:
stack1 = ["Z", "N"]
stack2 = ["M", "C", "D"]
stack3 = ["P"]
all_stacks = [stack1, stack2, stack3]
print_stacks(all_stacks)

    [D]    
[N] [C]    
[Z] [M] [P]
 1   2   3 


In [6]:
example_moves = parse_movements("./example_day05.txt",0)
example_moves

[(1, 2, 1), (3, 1, 3), (2, 2, 1), (1, 1, 2)]

In [7]:
for move in example_moves:
    execute_move_fifo(all_stacks, move)
print_stacks(all_stacks)

        [Z]
        [N]
        [D]
[C] [M] [P]
 1   2   3 


In [8]:
task1(all_stacks)

'CMZ'

### Real Input

```
[T]             [P]     [J]        
[F]     [S]     [T]     [R]     [B]
[V]     [M] [H] [S]     [F]     [R]
[Z]     [P] [Q] [B]     [S] [W] [P]
[C]     [Q] [R] [D] [Z] [N] [H] [Q]
[W] [B] [T] [F] [L] [T] [M] [F] [T]
[S] [R] [Z] [V] [G] [R] [Q] [N] [Z]
[Q] [Q] [B] [D] [J] [W] [H] [R] [J]
 1   2   3   4   5   6   7   8   9 

```

In [12]:
import re
def parse_setup(fn: str, no_header_lines: int) -> list[list[str]]:
    def parse_crates(stacks:list[list[str]], line: str, num_cols:int, col_size: int = 4):
        # parse columns
        for i in range(num_cols):
            idx = i*col_size
            substr = line[idx:idx+col_size]
            match = re.search(r"\[(\w)\]", substr)
            if match is not None:
                crate = match.group(1)
                stacks[i].append(crate)

    with open(fn, 'r') as f:
        input = f.readlines()
    # header only
    num_cols = len(input[no_header_lines].split())
    input = input[:no_header_lines]
    # create stacks
    stacks = [ list() for _ in range(num_cols)]

    for line in input:
        parse_crates(stacks, line, num_cols)        
    # reverse stacks
    for stack in stacks:
        stack = stack.reverse()
    return stacks


In [13]:
len_header = 8

In [14]:
stacks = parse_setup("./input_day05.txt",len_header)
stacks

[['Q', 'S', 'W', 'C', 'Z', 'V', 'F', 'T'],
 ['Q', 'R', 'B'],
 ['B', 'Z', 'T', 'Q', 'P', 'M', 'S'],
 ['D', 'V', 'F', 'R', 'Q', 'H'],
 ['J', 'G', 'L', 'D', 'B', 'S', 'T', 'P'],
 ['W', 'R', 'T', 'Z'],
 ['H', 'Q', 'M', 'N', 'S', 'F', 'R', 'J'],
 ['R', 'N', 'F', 'H', 'W'],
 ['J', 'Z', 'T', 'Q', 'P', 'R', 'B']]

In [15]:
print_stacks(stacks)

[T]             [P]     [J]        
[F]     [S]     [T]     [R]     [B]
[V]     [M] [H] [S]     [F]     [R]
[Z]     [P] [Q] [B]     [S] [W] [P]
[C]     [Q] [R] [D] [Z] [N] [H] [Q]
[W] [B] [T] [F] [L] [T] [M] [F] [T]
[S] [R] [Z] [V] [G] [R] [Q] [N] [Z]
[Q] [Q] [B] [D] [J] [W] [H] [R] [J]
 1   2   3   4   5   6   7   8   9 


In [16]:
moves = parse_movements("./input_day05.txt",len_header+2) # header + empty lines
moves[:3]

[(3, 8, 2), (3, 1, 5), (3, 1, 4)]

In [17]:
for move in moves:
    execute_move_fifo(stacks, move)
print_stacks(stacks)

                        [W]        
                        [P]        
                        [S]        
                        [D]        
                        [V]        
                        [R]        
                        [Q]        
                        [M]        
                        [Q]        
                        [F]        
                        [Z]        
                        [T]        
                        [Z]        
                        [D]        
                        [Z]        
                        [S]        
                        [F]        
                        [S]        
        [L]             [J]        
        [R]         [B] [N]        
        [J]         [R] [S]        
        [R]         [Q] [W]        
        [G]         [J] [N]        
        [T]     [H] [B] [T]        
        [M]     [R] [H] [P]     [F]
        [T] [V] [C] [B] [F]     [R]
[B] [Z] [H] [T] [W] [Q] [Q] [Q] [P]
 1   2   3   4   5   6   7  

In [18]:
task1(stacks)

'BZLVHBWQF'

## Part Two

As you watch the crane operator expertly rearrange the crates, you notice the process isn't following your prediction.

Some mud was covering the writing on the side of the crane, and you quickly wipe it away. The crane isn't a CrateMover 9000 - it's a _CrateMover 9001_.

The CrateMover 9001 is notable for many new and exciting features: air conditioning, leather seats, an extra cup holder, and _the ability to pick up and move multiple crates at once_.

Again considering the example above, the crates begin in the same configuration:

```
    [D]    
[N] [C]    
[Z] [M] [P]
 1   2   3 
```

Moving a single crate from stack 2 to stack 1 behaves the same as before:

```
[D]        
[N] [C]    
[Z] [M] [P]
 1   2   3 
```

However, the action of moving three crates from stack 1 to stack 3 means that those three moved crates _stay in the same order_, resulting in this new configuration:

```
        [D]
        [N]
    [C] [Z]
    [M] [P]
 1   2   3
```

Next, as both crates are moved from stack 2 to stack 1, they _retain their order_ as well:

```
        [D]
        [N]
[C]     [Z]
[M]     [P]
 1   2   3
```

Finally, a single crate is still moved from stack 1 to stack 2, but now it's crate `C` that gets moved:

```
        [D]
        [N]
        [Z]
[M] [C] [P]
 1   2   3
```

In this example, the CrateMover 9001 has put the crates in a totally different order: `_MCD_`.

Before the rearrangement process finishes, update your simulation so that the Elves know where they should stand to be ready to unload the final supplies. _After the rearrangement procedure completes, what crate ends up on top of each stack?_

In [20]:
def execute_move_lifo(stacks: list[list[str]], move: tuple[int,int,int]):
    num, from_, to_ = move
    # indices start at 1 so move -1
    from_ -= 1
    to_ -= 1
    cache = []
    # remove one after the other
    for _ in range(num):
        cache.append(stacks[from_].pop())
    # LIFO
    # add to new destination
    for _ in range(len(cache)):
        stacks[to_].append(cache.pop())
    return

In [21]:
stacks = parse_setup("./input_day05.txt",len_header)
moves = parse_movements("./input_day05.txt",len_header+2) # header + empty lines
print_stacks(stacks)

[T]             [P]     [J]        
[F]     [S]     [T]     [R]     [B]
[V]     [M] [H] [S]     [F]     [R]
[Z]     [P] [Q] [B]     [S] [W] [P]
[C]     [Q] [R] [D] [Z] [N] [H] [Q]
[W] [B] [T] [F] [L] [T] [M] [F] [T]
[S] [R] [Z] [V] [G] [R] [Q] [N] [Z]
[Q] [Q] [B] [D] [J] [W] [H] [R] [J]
 1   2   3   4   5   6   7   8   9 


In [22]:
for move in moves:
    execute_move_lifo(stacks, move)
print_stacks(stacks)

                        [Z]        
                        [R]        
                        [W]        
                        [H]        
                        [P]        
                        [C]        
                        [F]        
                        [W]        
                        [T]        
                        [H]        
                        [P]        
                        [R]        
                        [P]        
                        [N]        
                        [V]        
                        [T]        
                        [D]        
                        [T]        
        [G]             [Q]        
        [S]         [T] [Q]        
        [B]         [H] [S]        
        [V]         [R] [Z]        
        [R]         [Z] [J]        
        [R]     [Q] [J] [F]        
        [F]     [S] [B] [Q]     [L]
        [N] [J] [M] [Q] [M]     [Z]
[T] [D] [B] [R] [F] [W] [B] [S] [Q]
 1   2   3   4   5   6   7  

In [23]:
task1(stacks)

'TDGJQTZSL'